https://mlwhiz.com/blog/2019/02/19/siver_medal_kaggle_learnings/

https://towardsdatascience.com/https-medium-com-tanaygahlot-moving-beyond-the-distributional-model-for-word-representation-b0823f1769f8

In [0]:
from __future__ import absolute_import, division

import os
import time
import numpy as np
import pandas as pd
import gensim
from tqdm import tqdm
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from nltk.stem.lancaster import LancasterStemmer
lc = LancasterStemmer()
from nltk.stem import SnowballStemmer
sb = SnowballStemmer("english")
import gc

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate, LSTM, GRU
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

import sys
from os.path import dirname
#sys.path.append(dirname(dirname(__file__)))
from keras import initializers
from keras.engine import InputSpec, Layer
from keras import backend as K

import spacy


Using TensorFlow backend.


In [0]:
# https://github.com/bfelbo/DeepMoji/blob/master/deepmoji/attlayer.py
class AttentionWeightedAverage(Layer):
    """
    Computes a weighted average of the different channels across timesteps.
    Uses 1 parameter pr. channel to compute the attention value for a single timestep.
    """

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weights = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [0]:
# https://www.kaggle.com/cpmpml/spell-checker-using-word2vec
spell_model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/Colab Notebooks/wiki-news-300d-1M.vec')
words = spell_model.index2word
w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i
WORDS = w_rank

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def words(text):
   return re.findall(r'\w+', text.lower())
def P(word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)
def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or [word])
def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)
def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))
def singlify(word):
    return "".join([letter for i,letter in enumerate(word) if i == 0 or letter != word[i-1]])


In [0]:
# modified version of 
# https://www.kaggle.com/sudalairajkumar/a-look-at-different-embeddings
# https://www.kaggle.com/danofer/different-embeddings-with-attention-fork
# https://www.kaggle.com/shujian/different-embeddings-with-attention-fork-fork
def load_glove(word_dict, lemma_dict):
    EMBEDDING_FILE = '/content/drive/My Drive/Colab Notebooks/glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
    embed_size = 300
    nb_words = len(word_dict)+1
    embedding_matrix = np.zeros((nb_words, embed_size), dtype=np.float32)
    unknown_vector = np.zeros((embed_size,), dtype=np.float32) - 1.
    print(unknown_vector[:5])
    for key in tqdm(word_dict):
        word = key
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = key.lower()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = key.upper()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = key.capitalize()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = ps.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = lc.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = sb.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = lemma_dict[key]
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        if len(key) > 1:
            word = correction(key)
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[word_dict[key]] = embedding_vector
                continue
        embedding_matrix[word_dict[key]] = unknown_vector                    
    return embedding_matrix, nb_words 


def load_fasttext(word_dict, lemma_dict):
    EMBEDDING_FILE = '/content/drive/My Drive/Colab Notebooks/paragram_300_sl999.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)
    embed_size = 300
    nb_words = len(word_dict)+1
    embedding_matrix = np.zeros((nb_words, embed_size), dtype=np.float32)
    unknown_vector = np.zeros((embed_size,), dtype=np.float32) - 1.
    print(unknown_vector[:5])
    for key in tqdm(word_dict):
        word = key
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = key.lower()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = key.upper()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = key.capitalize()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = ps.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = lc.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = sb.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = lemma_dict[key]
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        if len(key) > 1:
            word = correction(key)
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[word_dict[key]] = embedding_vector
                continue
        embedding_matrix[word_dict[key]] = unknown_vector                    
    return embedding_matrix, nb_words 


def load_para(word_dict, lemma_dict):
    EMBEDDING_FILE = '/content/drive/My Drive/Colab Notebooks/paragram_300_sl999.txt'
    def get_coefs(word,*arr): 
      return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)
    embed_size = 300
    nb_words = len(word_dict)+1
    embedding_matrix = np.zeros((nb_words, embed_size), dtype=np.float32)
    unknown_vector = np.zeros((embed_size,), dtype=np.float32) - 1.
    print(unknown_vector[:5])
    for key in tqdm(word_dict):
        word = key
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = key.lower()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = key.upper()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = key.capitalize()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = ps.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = lc.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = sb.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = lemma_dict[key]
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        if len(key) > 1:
            word = correction(key)
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[word_dict[key]] = embedding_vector
                continue
        embedding_matrix[word_dict[key]] = unknown_vector                    
    return embedding_matrix, nb_words 


In [0]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [0]:
def build_model(embedding_matrix, nb_words, embedding_size=300):
    inp = Input(shape=(max_length,))
    x = Embedding(nb_words, embedding_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(0.3)(x)
    x1 = Bidirectional(CuDNNLSTM(256, return_sequences=True))(x)
    x2 = Bidirectional(CuDNNGRU(128, return_sequences=True))(x1)
    max_pool1 = GlobalMaxPooling1D()(x1)
    max_pool2 = GlobalMaxPooling1D()(x2)
    conc = Concatenate()([max_pool1, max_pool2])
    predictions = Dense(1, activation='sigmoid')(conc)
    model = Model(inputs=inp, outputs=predictions)
    adam = optimizers.Adam(lr=learning_rate)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [0]:
def model_lstm_atten(embedding_matrix, nb_words, embedding_size=300):
    inp = Input(shape=(max_length,))
    x = Embedding(nb_words, embedding_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [0]:
start_time = time.time()
print("Loading data ...")
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/train.csv').fillna(' ')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/test.csv').fillna(' ')
train_text = train['text']
test_text = test['text']
text_list = pd.concat([train_text, test_text])
y = train['target'].values
num_train_data = y.shape[0]
print("--- %s seconds ---" % (time.time() - start_time))


Loading data ...
--- 2.704399347305298 seconds ---


In [0]:
ids_with_target_error = [328,443,513,2619,3640,3900,4342,5781,6552,6554,6570,6701,6702,6729,6861,7226]
train[train['id'].isin(ids_with_target_error)]

,id,keyword,location,text,target
229,328,annihilated,,Ready to get annihilated for the BUCS game,1
301,443,apocalypse,,Short Reading\n\nApocalypse 21:1023 \n\nIn the...,1
356,513,army,Studio,But if you build an army of 100 dogs and their...,1
1822,2619,crashed,,My iPod crashed..... \n#WeLoveYouLouis \n#MTVH...,1
2536,3640,desolation,"Quilmes , Arg",This desperation dislocation\nSeparation conde...,1
2715,3900,devastated,PG Chillin!,Man Currensy really be talkin that talk... I'd...,1
3024,4342,dust%20storm,chicago,Going to a fest? Bring swimming goggles for th...,1
4068,5781,forest%20fires,,Campsite recommendations \nToilets /shower \nP...,1
4609,6552,injury,Saint Paul,My prediction for the Vikings game this Sunday...,1
4611,6554,injury,,Dante Exum's knee injury could stem Jazz's hop...,1


In [0]:
train.at[train['id'].isin(ids_with_target_error),'target'] = 0
train[train['id'].isin(ids_with_target_error)]

,id,keyword,location,text,target
229,328,annihilated,,Ready to get annihilated for the BUCS game,0
301,443,apocalypse,,Short Reading\n\nApocalypse 21:1023 \n\nIn the...,0
356,513,army,Studio,But if you build an army of 100 dogs and their...,0
1822,2619,crashed,,My iPod crashed..... \n#WeLoveYouLouis \n#MTVH...,0
2536,3640,desolation,"Quilmes , Arg",This desperation dislocation\nSeparation conde...,0
2715,3900,devastated,PG Chillin!,Man Currensy really be talkin that talk... I'd...,0
3024,4342,dust%20storm,chicago,Going to a fest? Bring swimming goggles for th...,0
4068,5781,forest%20fires,,Campsite recommendations \nToilets /shower \nP...,0
4609,6552,injury,Saint Paul,My prediction for the Vikings game this Sunday...,0
4611,6554,injury,,Dante Exum's knee injury could stem Jazz's hop...,0


In [0]:
import matplotlib.pyplot as plt
train['word_len']=train['text'].apply(lambda x:len(x))
test['word_len']=test['text'].apply(lambda x:len(x))
print("mean : ",train['word_len'].mean())
print("median : ",train['word_len'].median())
print("mean : ",test['word_len'].mean())
print("median : ",test['word_len'].median())
train['word_len']=train['text'].apply(lambda x:len(x))
# print(train['word_len'].quantile(0.10))
# print(train['word_len'].quantile(0.20))
# print(train['word_len'].quantile(0.30))
# print(train['word_len'].quantile(0.40))
# print(train['word_len'].quantile(0.50))
# print(train['word_len'].quantile(0.60))
# print(train['word_len'].quantile(0.70))
# print(train['word_len'].quantile(0.80))
# print(train['word_len'].quantile(0.90))

mean :  101.03743596479706
median :  107.0
mean :  102.10818265399939
median :  109.0


In [0]:
# !python -m spacy download en_core_web_lg

In [0]:
start_time = time.time()
print("Spacy NLP ...")
nlp = spacy.load('/content/drive/My Drive/Colab Notebooks/en_core_web_lg-2.1.0/en_core_web_lg/en_core_web_lg-2.1.0', disable=['parser','ner','tagger'])
nlp.vocab.add_flag(lambda s: s.lower() in spacy.lang.en.stop_words.STOP_WORDS, spacy.attrs.IS_STOP)
word_dict = {}
word_index = 1
lemma_dict = {}
docs = nlp.pipe(text_list, n_threads = 2)
word_sequences = []
for doc in tqdm(docs):
    word_seq = []
    for token in doc:
        if (token.text not in word_dict) and (token.pos_ is not "PUNCT"):
            word_dict[token.text] = word_index
            word_index += 1
            lemma_dict[token.text] = token.lemma_
        if token.pos_ is not "PUNCT":
            word_seq.append(word_dict[token.text])
    word_sequences.append(word_seq)
del docs
gc.collect()
train_word_sequences = word_sequences[:num_train_data]
test_word_sequences = word_sequences[num_train_data:]
print("--- %s seconds ---" % (time.time() - start_time))

Spacy NLP ...


10876it [00:04, 2313.99it/s]


--- 39.17442083358765 seconds ---


In [0]:
# hyperparameters
max_length = 140
embedding_size = 600
learning_rate = 0.001
batch_size = 512
num_epoch = 4

train_word_sequences = pad_sequences(train_word_sequences, maxlen=max_length, padding='post')
test_word_sequences = pad_sequences(test_word_sequences, maxlen=max_length, padding='post')
print(train_word_sequences[:10])
print(test_word_sequences[:10])
pred_prob = np.zeros((len(test_word_sequences),), dtype=np.float32)

[[  1   2   3 ...   0   0   0]
 [ 15  16  17 ...   0   0   0]
 [ 23  24  25 ...   0   0   0]
 ...
 [ 86  87  88 ...   0   0   0]
 [ 96  97  98 ...   0   0   0]
 [ 86  87 105 ...   0   0   0]]
[[  46  762   57 ...    0    0    0]
 [8964  461    8 ...    0    0    0]
 [ 630  108   57 ...    0    0    0]
 ...
 [1698  168  323 ...    0    0    0]
 [ 157   57 2501 ...    0    0    0]
 [7829  972  168 ...    0    0    0]]


In [0]:
start_time = time.time()
print("Loading embedding matrix ...")
embedding_matrix_glove, nb_words = load_glove(word_dict, lemma_dict)
embedding_matrix_fasttext, nb_words = load_fasttext(word_dict, lemma_dict)
embedding_matrix = np.concatenate((embedding_matrix_glove, embedding_matrix_fasttext), axis=1)
print("--- %s seconds ---" % (time.time() - start_time))

Loading embedding matrix ...


  3%|▎         | 886/34503 [00:00<00:03, 8840.83it/s]

[-1. -1. -1. -1. -1.]


  3%|▎         | 1175/34503 [00:00<00:02, 11719.83it/s]

[-1. -1. -1. -1. -1.]


100%|██████████| 34503/34503 [00:07<00:00, 4737.23it/s]


--- 244.89796924591064 seconds ---


In [0]:
start_time = time.time()
print("Start training ...")
model = model_lstm_atten(embedding_matrix, nb_words, embedding_size)
model.fit(train_word_sequences, y, batch_size=batch_size, epochs=num_epoch-1, verbose=2)
pred_prob += 0.15*np.squeeze(model.predict(test_word_sequences, batch_size=batch_size, verbose=2))
model.fit(train_word_sequences, y, batch_size=batch_size, epochs=1, verbose=2)
pred_prob += 0.35*np.squeeze(model.predict(test_word_sequences, batch_size=batch_size, verbose=2))
del model, embedding_matrix_fasttext, embedding_matrix
gc.collect()
K.clear_session()
print("--- %s seconds ---" % (time.time() - start_time))

Start training ...











Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/3
 - 10s - loss: 0.6110 - acc: 0.6585
Epoch 2/3
 - 3s - loss: 0.4416 - acc: 0.8034
Epoch 3/3
 - 3s - loss: 0.3975 - acc: 0.8265
Epoch 1/1
 - 3s - loss: 0.3635 - acc: 0.8467


--- 28.27223563194275 seconds ---


In [0]:
start_time = time.time()
print("Loading embedding matrix ...")
embedding_matrix_fasttext, nb_words = load_fasttext(word_dict, lemma_dict)
embedding_matrix_para, nb_words = load_para(word_dict, lemma_dict)
# embedding_matrix = np.concatenate((embedding_matrix_glove, embedding_matrix_para), axis=1)
embedding_matrix = np.concatenate((0.60*embedding_matrix_glove, 0.40*embedding_matrix_fasttext), axis=1)

print("--- %s seconds ---" % (time.time() - start_time))


Loading embedding matrix ...


  3%|▎         | 1087/34503 [00:00<00:03, 10853.91it/s]

[-1. -1. -1. -1. -1.]


  3%|▎         | 1198/34503 [00:00<00:02, 11972.25it/s]

[-1. -1. -1. -1. -1.]


100%|██████████| 34503/34503 [00:07<00:00, 4685.70it/s]


--- 26.496766328811646 seconds ---


In [0]:
embedding_matrix_fasttext.shape

(34504, 300)

In [0]:
# start_time = time.time()
# print("Start training ...")
model = model_lstm_atten(embedding_matrix, nb_words, embedding_size)
# model.fit(train_word_sequences, y, batch_size=batch_size, epochs=num_epoch-1, verbose=2)
# pred_prob += 0.15*np.squeeze(model.predict(test_word_sequences, batch_size=batch_size, verbose=2))
# model.fit(train_word_sequences, y, batch_size=batch_size, epochs=1, verbose=2)
# pred_prob += 0.35*np.squeeze(model.predict(test_word_sequences, batch_size=batch_size, verbose=2))
# print("--- %s seconds ---" % (time.time() - start_time))

In [0]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
# from sklearn import model_selection   
# kfold=model_selection.KFold
skf = StratifiedKFold(n_splits=4)
skf.get_n_splits(train_word_sequences, y)
cvscores = []
i=1
for train_1, test_1 in skf.split(train_word_sequences, y):
  model = model_lstm_atten(embedding_matrix_fasttext, nb_words, embedding_size=300)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# Fit the model
  print(model.fit(train_word_sequences[train_1], y[train_1], epochs=15, batch_size=512, verbose=2, validation_data=(train_word_sequences[test_1], y[test_1])))
	# evaluate the model
  # scores = model.evaluate(test_word_sequences[test], y[test], verbose=2)
  # print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  # cvscores.append(scores[1] * 100)
  print(i, "Fold Completed: ",   "--X-- "*25)
  i=i+1
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))












Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 5709 samples, validate on 1904 samples
Epoch 1/15
 - 10s - loss: 0.6354 - acc: 0.6173 - val_loss: 0.5404 - val_acc: 0.7574
Epoch 2/15
 - 3s - loss: 0.4991 - acc: 0.7709 - val_loss: 0.4778 - val_acc: 0.7925
Epoch 3/15
 - 3s - loss: 0.4278 - acc: 0.8135 - val_loss: 0.5211 - val_acc: 0.7784
Epoch 4/15
 - 3s - loss: 0.4119 - acc: 0.8203 - val_loss: 0.4732 - val_acc: 0.7889
Epoch 5/15
 - 3s - loss: 0.3765 - acc: 0.8417 - val_loss: 0.4707 - val_acc: 0.7994
Epoch 6/15
 - 3s - loss: 0.3608 - acc: 0.8497 - val_loss: 0.5251 - val_acc: 0.7700
Epoch 7/15
 - 3s - loss: 0.3455 - acc: 0.8537 - val_loss: 0.4933 - val_acc: 0.7878
Epoch 8/15
 - 3s - loss: 0.3188 - acc: 0.8707 - val_loss: 0.5292 - val_acc: 0.7826
Epoch 9/15
 - 4s - loss: 0.3080 - acc: 0.8749 - val_loss: 0.5032 - val_acc: 0.7857
Epoch 10/15
 - 4s - loss: 0.2810 - acc: 0.8879 - val_loss: 0.5512 - val_acc: 0.7763
Epoch 11/15
 

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [0]:
# original=pd.read_csv("/content/drive/My Drive/Colab Notebooks/NLP Original.csv")

In [0]:
# pred_prob=model.predict(test_word_sequences, batch_size=batch_size, verbose=2)
# f1_score(original['target'], submission['target'])

0.7324403789611239

In [0]:
submission = pd.DataFrame.from_dict({'id': test['id']})
pred_prob=model.predict(test_word_sequences, batch_size=batch_size, verbose=2)
submission['target'] = np.array(pred_prob.round())
submission.to_csv('submission.csv', index=False)
original=pd.read_csv("/content/drive/My Drive/Colab Notebooks/NLP Original.csv")
original['target'].value_counts()

0    1861
1    1402
Name: target, dtype: int64

In [0]:
submission['target'].value_counts()

0.0    2056
1.0    1207
Name: target, dtype: int64

In [0]:
embedding_size

600

In [0]:
def model_cnn(embedding_matrix):
    filter_sizes = [1,2,3,5]
    num_filters = 36
    embedding_size=300

    inp = Input(shape=(max_length,))
    x = Embedding(nb_words, 300, weights=[embedding_matrix], trainable=False)(inp)
    x = Reshape((max_length, 300, 1))(x)

    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], embedding_size),
                                     kernel_initializer='he_normal', activation='relu')(x)
        maxpool_pool.append(MaxPool2D(pool_size=(max_length - filter_sizes[i] + 1, 1))(conv))

    z = Concatenate(axis=1)(maxpool_pool)   
    z = Flatten()(z)
    z = Dropout(0.1)(z)

    outp = Dense(1, activation="sigmoid")(z)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [0]:
def model_lstm_atten(embedding_matrix, nb_words, embedding_size=300):
    inp = Input(shape=(max_length,))
    x = Embedding(nb_words, embedding_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

skf = StratifiedKFold(n_splits=4)
skf.get_n_splits(train_word_sequences, y)
cvscores = []
i=1
for train_1, test_1 in skf.split(train_word_sequences, y):
  model = model_cnn(embedding_matrix_fasttext)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# Fit the model
  print(model.fit(train_word_sequences[train_1], y[train_1], epochs=15, batch_size=512, verbose=2, validation_data=(train_word_sequences[test_1], y[test_1])))
	# evaluate the model
  # scores = model.evaluate(test_word_sequences[test], y[test], verbose=2)
  # print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  # cvscores.append(scores[1] * 100)
  print(i, "Fold Completed: ",   "--X-- "*25)
  i=i+1
# print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 5709 samples, validate on 1904 samples
Epoch 1/15
 - 4s - loss: 0.7283 - acc: 0.6045 - val_loss: 0.6005 - val_acc: 0.6896
Epoch 2/15
 - 1s - loss: 0.5359 - acc: 0.7322 - val_loss: 0.5213 - val_acc: 0.7600
Epoch 3/15
 - 1s - loss: 0.4601 - acc: 0.7807 - val_loss: 0.5089 - val_acc: 0.7652
Epoch 4/15
 - 1s - loss: 0.4187 - acc: 0.8115 - val_loss: 0.5053 - val_acc: 0.7631
Epoch 5/15
 - 1s - loss: 0.3851 - acc: 0.8320 - val_loss: 0.4887 - val_acc: 0.7778
Epoch 6/15
 - 1s - loss: 0.3604 - acc: 0.8487 - val_loss: 0.4868 - val_acc: 0.7694
Epoch 7/15
 - 1s - loss: 0.3405 - acc: 0.8592 - val_loss: 0.4869 - val_acc: 0.7784
Epoch 8/15
 - 1s - loss: 0.3223 - acc: 0.8699 - val_loss: 0.4929 - val_acc: 0.7742
Epoch 9/15
 - 1s - loss: 0.3058 - acc: 0.8823 - val_loss: 0.4971 - val_acc: 0.7684
Epoch 10/15
 - 1s - loss: 0.2884 - acc: 0.8875 - val_loss: 0.4918 - val_acc: 0.7705
Epoc

In [0]:
pred_prob=model.predict(test_word_sequences, batch_size=batch_size, verbose=2)
f1_score(original['target'], submission['target'])

0.7324403789611239

In [0]:
def model_lstm_atten(embedding_matrix, nb_words, embedding_size=300):
    inp = Input(shape=(max_length,))
    x = Embedding(nb_words, embedding_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

skf = StratifiedKFold(n_splits=4)
skf.get_n_splits(train_word_sequences, y)
cvscores = []
i=1
for train_1, test_1 in skf.split(train_word_sequences, y):
  model = model_lstm_atten(embedding_matrix, nb_words, embedding_size)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# Fit the model
  print(model.fit(train_word_sequences[train_1], y[train_1], epochs=15, batch_size=512, verbose=2, validation_data=(train_word_sequences[test_1], y[test_1])))
	# evaluate the model
  # scores = model.evaluate(test_word_sequences[test], y[test], verbose=2)
  # print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  # cvscores.append(scores[1] * 100)
  print(i, "Fold Completed: ",   "--X-- "*25)
  i=i+1
# print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Train on 5709 samples, validate on 1904 samples
Epoch 1/15
 - 4s - loss: 0.6294 - acc: 0.6534 - val_loss: 0.5754 - val_acc: 0.7264
Epoch 2/15
 - 2s - loss: 0.4953 - acc: 0.7753 - val_loss: 0.4601 - val_acc: 0.7878
Epoch 3/15
 - 2s - loss: 0.4344 - acc: 0.8106 - val_loss: 0.4548 - val_acc: 0.7899
Epoch 4/15
 - 2s - loss: 0.3977 - acc: 0.8271 - val_loss: 0.4514 - val_acc: 0.7946
Epoch 5/15
 - 2s - loss: 0.3782 - acc: 0.8418 - val_loss: 0.4451 - val_acc: 0.7967
Epoch 6/15
 - 2s - loss: 0.3645 - acc: 0.8481 - val_loss: 0.4432 - val_acc: 0.7988
Epoch 7/15
 - 2s - loss: 0.3640 - acc: 0.8445 - val_loss: 0.4601 - val_acc: 0.7904
Epoch 8/15
 - 2s - loss: 0.3445 - acc: 0.8576 - val_loss: 0.5145 - val_acc: 0.7831
Epoch 9/15
 - 2s - loss: 0.3436 - acc: 0.8564 - val_loss: 0.4649 - val_acc: 0.7925
Epoch 10/15
 - 2s - loss: 0.3237 - acc: 0.8671 - val_loss: 0.4582 - val_acc: 0.7889
Epoch 11/15
 - 2s - loss: 0.3070 - acc: 0.8758 - val_loss: 0.5133 - val_acc: 0.7899
Epoch 12/15
 - 2s - loss: 0.2926 - ac

In [0]:
pred_prob=model.predict(test_word_sequences, batch_size=batch_size, verbose=2)
f1_score(original['target'], submission['target'])

0.7389804522805673

In [0]:
def model_lstm_atten(embedding_matrix, nb_words, embedding_size=300):
    inp = Input(shape=(max_length,))
    x = Embedding(nb_words, embedding_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(LSTM(100, return_sequences=True))(x)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(48, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
embedding_matrix_fasttext, nb_words = load_fasttext(word_dict, lemma_dict)
embedding_matrix_para, nb_words = load_para(word_dict, lemma_dict)
# embedding_matrix = np.concatenate((embedding_matrix_glove, embedding_matrix_para), axis=1)
embedding_matrix = np.concatenate((0.60*embedding_matrix_glove, 0.40*embedding_matrix_fasttext), axis=1)

skf = StratifiedKFold(n_splits=4)
skf.get_n_splits(train_word_sequences, y)
cvscores = []
i=1
for train_1, test_1 in skf.split(train_word_sequences, y):
  model = model_lstm_atten(embedding_matrix, nb_words, embedding_size)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# Fit the model
  print(model.fit(train_word_sequences[train_1], y[train_1], epochs=15, batch_size=512, verbose=2, validation_data=(train_word_sequences[test_1], y[test_1])))
	# evaluate the model
  # scores = model.evaluate(test_word_sequences[test], y[test], verbose=2)
  # print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  # cvscores.append(scores[1] * 100)
  print(i, "Fold Completed: ",   "--X-- "*25)
  i=i+1
# print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

  4%|▎         | 1271/34498 [00:00<00:02, 12698.24it/s]

[-1. -1. -1. -1. -1.]


  4%|▍         | 1318/34498 [00:00<00:02, 13112.14it/s]

[-1. -1. -1. -1. -1.]


100%|██████████| 34498/34498 [00:05<00:00, 6138.60it/s]


Train on 5709 samples, validate on 1904 samples
Epoch 1/15
 - 25s - loss: 0.6476 - acc: 0.6085 - val_loss: 0.5740 - val_acc: 0.7405
Epoch 2/15
 - 13s - loss: 0.5045 - acc: 0.7700 - val_loss: 0.4661 - val_acc: 0.7931
Epoch 3/15
 - 13s - loss: 0.4295 - acc: 0.8143 - val_loss: 0.4710 - val_acc: 0.7899
Epoch 4/15
 - 13s - loss: 0.3958 - acc: 0.8285 - val_loss: 0.4510 - val_acc: 0.7915
Epoch 5/15
 - 13s - loss: 0.3775 - acc: 0.8425 - val_loss: 0.4482 - val_acc: 0.7920
Epoch 6/15
 - 13s - loss: 0.3621 - acc: 0.8471 - val_loss: 0.4568 - val_acc: 0.7931
Epoch 7/15
 - 13s - loss: 0.3476 - acc: 0.8550 - val_loss: 0.4846 - val_acc: 0.7868
Epoch 8/15
 - 13s - loss: 0.3443 - acc: 0.8609 - val_loss: 0.4856 - val_acc: 0.7889
Epoch 9/15
 - 13s - loss: 0.3230 - acc: 0.8653 - val_loss: 0.4674 - val_acc: 0.7957
Epoch 10/15
 - 13s - loss: 0.3072 - acc: 0.8800 - val_loss: 0.5021 - val_acc: 0.7889
Epoch 11/15
 - 13s - loss: 0.2895 - acc: 0.8847 - val_loss: 0.5177 - val_acc: 0.7868
Epoch 12/15
 - 12s - loss:

In [0]:
pred_prob=model.predict(test_word_sequences, batch_size=batch_size, verbose=2)
f1_score(original['target'], submission['target'])

0.7389804522805673

In [0]:
def model_lstm_atten(embedding_matrix, nb_words, embedding_size=600):
    inp = Input(shape=(max_length,))
    x = Embedding(nb_words, embedding_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(LSTM(100, return_sequences=True))(x)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(48, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adagrad', metrics=['accuracy'])
    return model
# embedding_matrix_fasttext, nb_words = load_fasttext(word_dict, lemma_dict)
# embedding_matrix_para, nb_words = load_para(word_dict, lemma_dict)
# embedding_matrix = np.concatenate((embedding_matrix_glove, embedding_matrix_para), axis=1)
embedding_matrix = np.concatenate((0.20*embedding_matrix_glove, 0.80*embedding_matrix_fasttext), axis=1)

skf = StratifiedKFold(n_splits=4)
skf.get_n_splits(train_word_sequences, y)
cvscores = []
i=1
for train_1, test_1 in skf.split(train_word_sequences, y):
  model = model_lstm_atten(embedding_matrix, nb_words, embedding_size)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# Fit the model
  print(model.fit(train_word_sequences[train_1], y[train_1], epochs=15, batch_size=512, verbose=2, validation_data=(train_word_sequences[test_1], y[test_1])))
	# evaluate the model
  # scores = model.evaluate(test_word_sequences[test], y[test], verbose=2)
  # print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  # cvscores.append(scores[1] * 100)
  print(i, "Fold Completed: ",   "--X-- "*25)
  i=i+1
# print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Train on 5709 samples, validate on 1904 samples
Epoch 1/15
 - 28s - loss: 0.6540 - acc: 0.6045 - val_loss: 0.6180 - val_acc: 0.7190
Epoch 2/15
 - 13s - loss: 0.5224 - acc: 0.7683 - val_loss: 0.4909 - val_acc: 0.7621
Epoch 3/15
 - 13s - loss: 0.4490 - acc: 0.8014 - val_loss: 0.4744 - val_acc: 0.7747
Epoch 4/15
 - 13s - loss: 0.4069 - acc: 0.8252 - val_loss: 0.4827 - val_acc: 0.7784
Epoch 5/15
 - 13s - loss: 0.4061 - acc: 0.8280 - val_loss: 0.4709 - val_acc: 0.7836
Epoch 6/15
 - 13s - loss: 0.3820 - acc: 0.8332 - val_loss: 0.4546 - val_acc: 0.7978
Epoch 7/15
 - 13s - loss: 0.3602 - acc: 0.8506 - val_loss: 0.4657 - val_acc: 0.7952
Epoch 8/15
 - 13s - loss: 0.3509 - acc: 0.8504 - val_loss: 0.4826 - val_acc: 0.7805
Epoch 9/15
 - 13s - loss: 0.3270 - acc: 0.8628 - val_loss: 0.4792 - val_acc: 0.7973
Epoch 10/15
 - 13s - loss: 0.3082 - acc: 0.8751 - val_loss: 0.4975 - val_acc: 0.7889
Epoch 11/15
 - 13s - loss: 0.2869 - acc: 0.8840 - val_loss: 0.5437 - val_acc: 0.7873
Epoch 12/15
 - 13s - loss:

In [0]:
pred_prob=model.predict(test_word_sequences, batch_size=batch_size, verbose=2)
f1_score(original['target'], submission['target'])

In [0]:
embedding_matrix_para.shape

(34504, 300)

In [0]:
def model_lstm_atten(embedding_matrix, nb_words, embedding_size=300):
    inp = Input(shape=(max_length,))
    x = Embedding(nb_words, embedding_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(CuDNNLSTM(100, return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(48, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# embedding_matrix_fasttext, nb_words = load_fasttext(word_dict, lemma_dict)
# embedding_matrix_para, nb_words = load_para(word_dict, lemma_dict)
# embedding_matrix = np.concatenate((embedding_matrix_glove, embedding_matrix_para), axis=1)
# embedding_matrix = np.concatenate((0.60*embedding_matrix_glove, 0.40*embedding_matrix_fasttext), axis=1)

skf = StratifiedKFold(n_splits=4)
skf.get_n_splits(train_word_sequences, y)
cvscores = []
i=1
for train_1, test_1 in skf.split(train_word_sequences, y):
  model = model_lstm_atten(embedding_matrix_glove, nb_words, embedding_size=300)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# Fit the model
  print(model.fit(train_word_sequences[train_1], y[train_1], epochs=15, batch_size=512, verbose=2, validation_data=(train_word_sequences[test_1], y[test_1])))
	# evaluate the model
  # scores = model.evaluate(test_word_sequences[test], y[test], verbose=2)
  # print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  # cvscores.append(scores[1] * 100)
  print(i, "Fold Completed: ",   "--X-- "*25)
  i=i+1
# print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Train on 5709 samples, validate on 1904 samples
Epoch 1/15
 - 7s - loss: 0.6169 - acc: 0.6637 - val_loss: 0.5010 - val_acc: 0.7700
Epoch 2/15
 - 3s - loss: 0.4579 - acc: 0.7996 - val_loss: 0.4568 - val_acc: 0.7946
Epoch 3/15
 - 3s - loss: 0.4062 - acc: 0.8224 - val_loss: 0.4432 - val_acc: 0.8025
Epoch 4/15
 - 3s - loss: 0.3863 - acc: 0.8352 - val_loss: 0.4512 - val_acc: 0.8015
Epoch 5/15
 - 3s - loss: 0.3677 - acc: 0.8439 - val_loss: 0.4606 - val_acc: 0.7988
Epoch 6/15
 - 3s - loss: 0.3551 - acc: 0.8511 - val_loss: 0.4511 - val_acc: 0.7978
Epoch 7/15
 - 3s - loss: 0.3350 - acc: 0.8592 - val_loss: 0.4541 - val_acc: 0.7910
Epoch 8/15
 - 3s - loss: 0.3212 - acc: 0.8681 - val_loss: 0.4920 - val_acc: 0.7941
Epoch 9/15
 - 3s - loss: 0.3015 - acc: 0.8734 - val_loss: 0.5312 - val_acc: 0.7878
Epoch 10/15
 - 3s - loss: 0.3116 - acc: 0.8697 - val_loss: 0.4748 - val_acc: 0.7920
Epoch 11/15
 - 3s - loss: 0.2745 - acc: 0.8912 - val_loss: 0.5148 - val_acc: 0.7826
Epoch 12/15
 - 3s - loss: 0.2556 - ac

In [0]:
def model_lstm_atten(embedding_matrix, nb_words, embedding_size=300):
    inp = Input(shape=(max_length,))
    x = Embedding(nb_words, embedding_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(LSTM(100, return_sequences=True))(x)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(48, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adagrad', metrics=['accuracy'])
    return model
# embedding_matrix_fasttext, nb_words = load_fasttext(word_dict, lemma_dict)
# embedding_matrix_para, nb_words = load_para(word_dict, lemma_dict)
# embedding_matrix = np.concatenate((embedding_matrix_glove, embedding_matrix_para), axis=1)
# embedding_matrix = np.concatenate((0.20*embedding_matrix_glove, 0.80*embedding_matrix_fasttext), axis=1)

skf = StratifiedKFold(n_splits=4)
skf.get_n_splits(train_word_sequences, y)
cvscores = []
i=1
for train_1, test_1 in skf.split(train_word_sequences, y):
  model = model_lstm_atten(embedding_matrix_para, nb_words, embedding_size)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# Fit the model
  print(model.fit(train_word_sequences[train_1], y[train_1], epochs=15, batch_size=512, verbose=2, validation_data=(train_word_sequences[test_1], y[test_1])))
	# evaluate the model
  # scores = model.evaluate(test_word_sequences[test], y[test], verbose=2)
  # print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  # cvscores.append(scores[1] * 100)
  print(i, "Fold Completed: ",   "--X-- "*25)
  i=i+1
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [0]:
pred_prob=model.predict(test_word_sequences, batch_size=batch_size, verbose=2)
f1_score(original['target'], submission['target'])

# BERT

In [0]:
!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo



Cloning into 'bert_repo'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 300.28 KiB | 566.00 KiB/s, done.
Resolving deltas: 100% (185/185), done.


In [0]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 2.3MB/s 


In [0]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm

import math
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
%matplotlib inline
import numpy as np
from bert.tokenization import FullTokenizer
from tqdm import tqdm_notebook
from tensorflow.keras import backend as K

# Initialize session
sess = tf.Session()

# Params for bert model and tokenization
bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
max_seq_length = 160

In [0]:
from keras.layers import Layer

class BertLayer(Layer):
    def __init__(self, n_fine_tune_layers=10, **kwargs):
        self.n_fine_tune_layers = n_fine_tune_layers
        self.trainable = True
        self.output_size = 768
        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            bert_path,
            trainable=self.trainable,
            name="{}_module".format(self.name)
        )
        trainable_vars = self.bert.variables
        
        # Remove unused layers
        trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]
        
        # Select how many layers to fine tune
        trainable_vars = trainable_vars[-self.n_fine_tune_layers :]
        
        # Add to trainable weights
        for var in trainable_vars:
            self._trainable_weights.append(var)
        
        # Add non-trainable weights
        for var in self.bert.variables:
            if var not in self._trainable_weights:
                self._non_trainable_weights.append(var)
        
        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
            "pooled_output"
        ]
        return result

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)

In [0]:
# Build model
def build_model(max_seq_length): 
    in_id = Input(shape=(max_seq_length,), name="input_ids")
    in_mask = Input(shape=(max_seq_length,), name="input_masks")
    in_segment = Input(shape=(max_seq_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]
    
    bert_output = BertLayer(n_fine_tune_layers=3)(bert_inputs)
    dense = Dense(256, activation='relu')(bert_output)
    pred = Dense(1, activation='sigmoid')(dense)
    
    model = Model(inputs=bert_inputs, outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)
from sklearn.model_selection import train_test_split

In [0]:
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
  When running eval/predict on the TPU, we need to pad the number of examples
  to be a multiple of the batch size, because the TPU requires a fixed batch
  size. The alternative is to drop the last batch, which is bad because it means
  the entire output data won't be generated.
  We use this class instead of `None` because treating `None` as padding
  battches could cause silent errors.
  """

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    bert_module =  hub.Module(bert_path)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    vocab_file, do_lower_case = sess.run(
        [
            tokenization_info["vocab_file"],
            tokenization_info["do_lower_case"],
        ]
    )

    return FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

def convert_single_example(tokenizer, example, max_seq_length=256):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        input_ids = [0] * max_seq_length
        input_mask = [0] * max_seq_length
        segment_ids = [0] * max_seq_length
        label = 0
        return input_ids, input_mask, segment_ids, label

    tokens_a = tokenizer.tokenize(example.text_a)
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0 : (max_seq_length - 2)]

    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    return input_ids, input_mask, segment_ids, example.label

def convert_examples_to_features(tokenizer, examples, max_seq_length=256):
    """Convert a set of `InputExample`s to a list of `InputFeatures`."""

    input_ids, input_masks, segment_ids, labels = [], [], [], []
    for example in tqdm_notebook(examples, desc="Converting examples to features"):
        input_id, input_mask, segment_id, label = convert_single_example(
            tokenizer, example, max_seq_length
        )
        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
        labels.append(label)
    return (
        np.array(input_ids),
        np.array(input_masks),
        np.array(segment_ids),
        np.array(labels).reshape(-1, 1),
    )

def convert_text_to_examples(texts, labels):
    """Create InputExamples"""
    InputExamples = []
    for text, label in zip(texts, labels):
        InputExamples.append(
            InputExample(guid=None, text_a=" ".join(text), text_b=None, label=label)
        )
    return InputExamples

def convert_text_to_examples_test(texts):
  InputExamples = []
  for text in texts:
    InputExamples.append(InputExample(guid=None, text_a=" ".join(text), text_b=None))
    
  return InputExamples
def convert_examples_to_features_test(tokenizer, examples, max_seq_length=160):
    """Convert a set of `InputExample`s to a list of `InputFeatures`."""

    input_ids, input_masks, segment_ids = [], [], []
    for example in tqdm_notebook(examples, desc="Converting examples to features"):
        input_id, input_mask, segment_id, label = convert_single_example(
            tokenizer, example, max_seq_length
        )
        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
        
    return (
        np.array(input_ids),
        np.array(input_masks),
        np.array(segment_ids),
           )

In [0]:
# Instantiate tokenizer
tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
train_text, test_text, train_label, test_label = train_test_split( train['text'], train['target'], test_size=0.20, random_state=42)

In [0]:
# Convert data to InputExample format
train_examples = convert_text_to_examples(train_text, train_label)
test_examples = convert_text_to_examples(test_text, test_label)

# Convert to features
(train_input_ids, train_input_masks, train_segment_ids, train_labels
 ) = convert_examples_to_features(tokenizer, train_examples, max_seq_length=max_seq_length)
(test_input_ids, test_input_masks, test_segment_ids, test_labels
) = convert_examples_to_features(tokenizer, test_examples, max_seq_length=max_seq_length)

In [0]:
test_submission_examples=convert_text_to_examples_test(test_df['text'])
(test_input_ids_submisson, test_input_masks_submisson, test_segment_ids_submisson
) = convert_examples_to_features_test(tokenizer, test_submission_examples, max_seq_length=max_seq_length)

In [0]:
in_id = Input(shape=(max_seq_length,), name="input_ids")
in_mask = Input(shape=(max_seq_length,), name="input_masks")
in_segment = Input(shape=(max_seq_length,), name="segment_ids")
bert_inputs = [in_id, in_mask, in_segment]
bert_output = BertLayer(n_fine_tune_layers=3)(bert_inputs)
dense = Dense(256, activation='relu')(bert_output)
pred = Dense(1, activation='sigmoid')(dense)
    
model = Model(inputs=bert_inputs, outputs=pred)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

NameError: ignored

In [0]:
history=model.fit(
    [train_input_ids, train_input_masks, train_segment_ids], 
    train_labels,
    validation_data=([test_input_ids, test_input_masks, test_segment_ids], test_labels),
    epochs=5,
    batch_size=128 )

NameError: ignored